In [1]:
import os
import pickle
import sys
import h5py
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from tqdm import tnrange, tqdm_notebook

# os.chdir('codes')
import models
import sklearn.metrics as metrics
from PIL import Image

import matplotlib.pyplot as plt
import torchvision.transforms as transforms
%matplotlib inline

In [2]:
data_path = 'C:/Users/admin/Desktop/pytorch_ovo/data/' # the path of the 'npc_v4_data.h5' file

In [3]:
with h5py.File(os.path.join(data_path, 'npc_v4_data.h5')) as hf:
    print(hf.keys())
    images_n = np.array(hf['images']['naturalistic'])
    neural_n = np.array(hf['neural']['naturalistic']['monkey_m']['stretch']['session_2'])

print(neural_n.shape)
n_images = neural_n.shape[1]
n_neurons = neural_n.shape[2]
size_imags = images_n.shape[0]
insp_layer = 'conv3'



KeysView(<HDF5 file "npc_v4_data.h5" (mode r+)>)
(36, 640, 52)


In [4]:
img = np.zeros((images_n.shape[0], images_n[0].shape[0], images_n[0].shape[1], 3))
for i in range(n_images):
    rgb_img = np.zeros((images_n[i].shape[0], images_n[i].shape[1], 3), dtype=np.uint8)
    rgb_img[:, :, 0] = images_n[i]
    rgb_img[:, :, 1] = images_n[i]
    rgb_img[:, :, 2] = images_n[i]
    img[i] = rgb_img

device = 'cuda:0' # device where you put your data and models
alexnet = models.alexnet(pretrained=True)
alexnet.to(device)
alexnet.eval()
for param in alexnet.parameters():
    param.requires_grad_(False)


data_x = np.transpose(img, (0, 3, 1, 2))
print(img.shape, data_x.shape)
neurons = neural_n[1]
x = torch.from_numpy(data_x[0:1]).float().to(device)
fmap = alexnet(x, layer=insp_layer)
print(fmap.shape)
sizes = fmap.shape[2:]



(640, 299, 299, 3) (640, 3, 299, 299)
torch.Size([1, 384, 17, 17])


In [13]:
device = 'cuda:0' # device where you put your data and models
data_path = './' # the path of the 'npc_v4_data.h5' file
batch_size = 50 # the batch size of the data loader
insp_layer = 'conv3' # the middle layer extracted from alexnet, available in {'conv1', 'conv2', 'conv3', 'conv4', 'conv5'}
reps = neural_n.shape[0]
rand_ind = np.arange(reps)
np.random.shuffle(rand_ind)


data_y_train = neural_n[:,:576].mean(0).astype(np.float32)
data_y_val_origin = neural_n[:, 576:].astype(np.float32)
data_y_val = data_y_val_origin.mean(0)

#data_x = images_n[:, np.newaxis].astype(np.float32)
data_x = img
print('images_n', images_n.shape)
print(data_x.shape)

data_x = data_x / 255 # (640, 1, 299, 299)

print(type(data_x))

print(data_x.shape)
#data_x = np.tile(data_x, [1, 3, 1, 1])
data_x = np.transpose(data_x, (0, 3, 1, 2))
print('data_x', data_x.shape)
data_x_train = data_x[:576]
data_x_val = data_x[576:]


class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_x, data_y):
        self.data_x = data_x
        self.data_y = data_y
    def __getitem__(self, index):
        return index, self.data_x[index], self.data_y[index]
    def __len__(self):
        return self.data_x.shape[0]

imagenet_mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32).view(1, 3, 1, 1).to(device)
imagenet_std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32).view(1, 3, 1, 1).to(device)
transform = lambda x : (x - imagenet_mean) / imagenet_std

dataset_train = Dataset(data_x_train, data_y_train)
dataset_val = Dataset(data_x_val, data_y_val)

loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

images_n (640, 299, 299)
(640, 299, 299, 3)
<class 'numpy.ndarray'>
(640, 299, 299, 3)
data_x (640, 3, 299, 299)


In [27]:
feature_map = torch.Tensor(n_images, fmap.shape[1], fmap.shape[2], fmap.shape[3])
feature_map.to(device)
print(feature_map.shape)
for i in range(n_images):
    x = torch.from_numpy(data_x[i:i + 1]).float().to(device)
    fmap = alexnet(x, layer = insp_layer)
    feature_map[i] = fmap

torch.Size([640, 384, 17, 17])


In [28]:
# CHOOSE THE AUGMENTS IF NECESSARY
lamd_s, lamd_d = [1e-1, 1e-1] # the coefficients of the losses. Try other coefficients!
epoches = 100 # total epochs for training the encoder
lr = 1e-1 # the learing rate for training the encoder

alexnet = models.alexnet(pretrained=True)
alexnet.to(device)
alexnet.eval()
for param in alexnet.parameters():
    param.requires_grad_(False)

x = torch.from_numpy(data_x[0:1]).float().to(device)
print("x:", x.shape)
fmap = alexnet(x, layer=insp_layer)

neurons = data_y_train.shape[1]
sizes = fmap.shape[2:]
print("fmap: ", fmap.shape)
print("size: ", sizes)
channels = fmap.shape[1]
print(neurons, sizes)
w_s = nn.Parameter(torch.randn(size=(neurons,) + sizes))
print(w_s.shape)

x: torch.Size([1, 3, 299, 299])
fmap:  torch.Size([1, 384, 17, 17])
size:  torch.Size([17, 17])
52 torch.Size([17, 17])
torch.Size([52, 17, 17])


In [29]:
class conv_encoder(nn.Module):

    def __init__(self, neurons, sizes, channels):
        super(conv_encoder, self).__init__()
        # PUT YOUR CODES HERE
        self.W_s = nn.Parameter(torch.randn(size=(neurons,) + sizes))
        self.W_d = nn.Parameter(torch.randn(size = (neurons,channels,1,1)))
        self.W_b = nn.Parameter(torch.randn(size = (1,neurons)))


    def forward(self, x):
        # PUT YOUR CODES HERE
        out = torch.einsum('bchw , nhw -> bnchw',x,self.W_s) # dimension : N,n,C,h,w
        out = torch.stack(
            [F.conv2d(out[:,n,:,:,:],torch.unsqueeze(self.W_d[n],0)) for n in range(neurons)],dim=1)
            #dimension:N,n,1,h,w
        out = torch.sum(out,dim=(2,3,4))
        out = out + self.W_b
        return out

def L_e(y,pred):
    return torch.mean(torch.sqrt(torch.sum((y-pred)**2,dim=1)))

def L_2(W_s,W_d,lamd_s=lamd_s,lamd_d=lamd_d):
    return lamd_s * torch.sum(W_s**2) + lamd_d * torch.sum(W_d**2)

K = torch.tensor([
    [0,-1,0],
    [-1,4,-1],
    [0,-1,0]],dtype=torch.float).to(device)
def L_laplace(W_s,lamd_s=lamd_s):
    return lamd_s * torch.sum(F.conv2d(torch.unsqueeze(W_s,1),K.unsqueeze(0).unsqueeze(0))**2)


#encoder = conv_encoder(neurons, sizes, channels).to(device)
encoder = conv_encoder(neurons, sizes, channels).to(device)

In [30]:
for i,(z, x, y) in enumerate(loader_train):
    print(z)
    break

tensor([460, 285,  44, 122,  83,  61, 522, 359,  20, 562, 565, 260, 284, 507,
         37, 543, 449, 493, 361, 453, 420, 242, 255, 563, 253, 249, 411, 138,
        295,  30, 109, 172,  18, 327, 272, 525,  13,  15, 317, 560, 140, 186,
        550,   1, 124, 162, 450, 205, 429, 381])


In [35]:
def train_model(encoder, optimizer):
    losses = []
    encoder.train()
    for i,(z, x,y) in enumerate(loader_train):
        optimizer.zero_grad()
        x = x.float().to(device)
        y = y.float().to(device)
        z = z.to(device)
        x = transform(x)
        fmap = feature_map[z - 1].to(device)
        out = encoder(fmap)
        l_e = L_e(y,out)
        l_2 = L_2(encoder.W_s,encoder.W_d)
        l_l = L_laplace(encoder.W_s)
#         print(f'L_e = {l_e} , L_2 = {l_2} , L_l = {l_l}')
        loss = L_e(y,out) + L_2(encoder.W_s,encoder.W_d) + L_laplace(encoder.W_s)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
#         print(f'iteration {i}, train loss: {losses[-1]}')

    return losses

def validate_model(encoder):
    encoder.eval()
    y_pred = []
    y_true = []
    losses = []
    for i,(z, x,y) in enumerate(loader_val):
        x = x.float().to(device)
        y = y.float().to(device)
        z = z.to(device)
        x = transform(x)
        fmap = feature_map[z - 1].to(device)
        out = encoder(fmap)
        y_pred.append(out)
        y_true.append(y)
        l_e = L_e(y,out)
        l_2 = L_2(encoder.W_s,encoder.W_d)
        l_l = L_laplace(encoder.W_s)
        print(f'L_e = {l_e} , L_2 = {l_2} , L_l = {l_l}')
        loss = L_e(y,out) + L_2(encoder.W_s,encoder.W_d) + L_laplace(encoder.W_s)
        losses.append(loss.item())
    y_pred = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    explained_variance = metrics.explained_variance_score(y_true = y_true.detach().cpu().numpy(),y_pred = y_pred.detach().cpu().numpy())
    return explained_variance,sum(losses)/len(losses)

"""
    You need to define the conv_encoder() class and train the encoder.
    The code of alexnet has been slightly modified from the torchvision, for convenience
    of extracting the middle layers.

    Example:
        >>> x = x.to(device) # x is a batch of images
        >>> x = transform(x)
        >>> fmap = alexnet(x, layer=insp_layer)
        >>> out= encoder(fmap)
        >>> ...
"""

'\n    You need to define the conv_encoder() class and train the encoder.\n    The code of alexnet has been slightly modified from the torchvision, for convenience\n    of extracting the middle layers.\n\n    Example:\n        >>> x = x.to(device) # x is a batch of images\n        >>> x = transform(x)\n        >>> fmap = alexnet(x, layer=insp_layer)\n        >>> out= encoder(fmap)\n        >>> ...\n'

In [36]:
# losses_train = []
# losses_val = []
# EVs = []

losses_train = []
losses_val = []
EVs = []

In [37]:

lr = 1e-1
optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)

In [38]:

for epoch in tqdm_notebook(range(epoches)):
    losses_train += train_model(encoder,optimizer)
    ev,loss = validate_model(encoder)
    EVs.append(ev)
    losses_val.append(loss)
    print(f'epoch {epoch}, EV = {ev}, val  loss = {loss} , train loss {sum(losses_train[-10:])/10}')



C:\Users\admin\anaconda3\envs\neural_control\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/100 [00:00<?, ?it/s]

L_e = 1503.5555419921875 , L_2 = 1967.69580078125 , L_l = 3312.328857421875
L_e = 1517.6104736328125 , L_2 = 1967.69580078125 , L_l = 3312.328857421875
epoch 0, EV = -62782.504845252406, val  loss = 6790.607421875 , train loss 14083.74345703125
L_e = 643.3994750976562 , L_2 = 1225.086669921875 , L_l = 973.2647705078125
L_e = 619.8322143554688 , L_2 = 1225.086669921875 , L_l = 973.2647705078125
epoch 1, EV = -13038.190079909105, val  loss = 2829.96728515625 , train loss 4252.468896484375
L_e = 305.5891418457031 , L_2 = 793.1348876953125 , L_l = 285.9554138183594
L_e = 276.92645263671875 , L_2 = 793.1348876953125 , L_l = 285.9554138183594
epoch 2, EV = -2854.688223031851, val  loss = 1370.3480834960938 , train loss 1941.6661376953125
L_e = 137.84507751464844 , L_2 = 534.4629516601562 , L_l = 90.95877838134766
L_e = 139.381591796875 , L_2 = 534.4629516601562 , L_l = 90.95877838134766
epoch 3, EV = -598.4357815522415, val  loss = 764.0350952148438 , train loss 1018.3526428222656
L_e = 72.1

In [41]:
print(max(EVs))

-0.4156657010316849
